In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Pheme.csv')
X = df['Text']
y = df['Label']

X.head(), y.head()

(0    #BREAKING: A Germanwings Airbus A320 has crash...
 1    Updated numbers @AP: BREAKING: #Germanwings CE...
 2    @YanniKouts @germanwings @flightradar24 A bit ...
 3    #Germanwings co-pilot suffered serious depress...
 4    @ThisIsGaZa إِنَّا لِلّهِ وَإِنَّـا إِلَيْهِ ر...
 Name: Text, dtype: object,
 0    rumours
 1    rumours
 2    rumours
 3    rumours
 4    rumours
 Name: Label, dtype: object)

In [13]:
type(y)

pandas.core.series.Series

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=2)

In [22]:
len(X_train), len(y_train)

(5140, 5140)

In [23]:
len(X_test), len(y_test)

(1286, 1286)

In [24]:
type(y_test)

pandas.core.series.Series

In [25]:
def label(x):
    if x == 'rumours':
        x = 1
    elif x == 'non-rumours':
        x = 0
    return x

y_test = y_test.apply(lambda x: label(x))
y_test

3408    0
577     1
3077    0
6308    1
2558    0
       ..
179     1
3291    0
2435    0
5750    0
6188    0
Name: Label, Length: 1286, dtype: int64

In [30]:
import torch

torch.tensor(y_test.values)

tensor([0, 1, 0,  ..., 0, 0, 0])

In [31]:
y = y.apply(lambda x: label(x))
torch.tensor(y.values)

tensor([1., 1., 1.,  ..., 0., 0., 0.], dtype=torch.float64)